In [3]:
import os
# import psutil
# import time
from dotenv import load_dotenv

from operations.stats_monitor import *
# from database.models import Base, PcStats, GpuStats, to_dict # Import your models and to_dict
# from operations.models import PcStatsCreateData, GpuStatsCreateData # Import your Pydantic models
from database.db_interface import DBInterface
from database.engine import init_db
load_dotenv('.env')
CONN_STRING = os.getenv("CONN_STRING")
await init_db(CONN_STRING)
# PcStatsDBInterface = DBInterface(PcStats)
# GpuStatsDBInterface = DBInterface(GpuStats)

Database 'pc_stats' already exists.
Ensuring database tables exist...
Database tables checked/created.
Asynchronous database initialization complete.


In [4]:
a = get_pc_stats()

In [5]:
a

[{'pc_usage': 0.4,
  'pc_freq': 3696.0,
  'ram_usage': 1.5,
  'ram_available': 25.4,
  'internet_receive': 119.38,
  'internet_sent': 73.85},
 {'gpu_id': 0, 'ram_usage': 2921.0, 'ram_available': 5037.0, 'temp': 50.0},
 {'gpu_id': 1, 'ram_usage': 6262.0, 'ram_available': 1806.0, 'temp': 43.0}]

In [6]:
await insert_stats_to_db(a)

Inserting PC stats...
Inserting GPU stats for GPU ID: 0...
Inserting GPU stats for GPU ID: 1...
1 PC stats entry inserted successfully.
2 GPU stats entries inserted successfully.


In [21]:
await delete_all_tables()

Deleting table: gpu_stats...
Successfully deleted table: gpu_stats
Deleting table: pc_stats...
Successfully deleted table: pc_stats
Tables 'gpu_stats','pc_stats' deleted.


In [4]:
from database.engine import AsyncDBSession
from sqlalchemy import text, select, update
async def open_async_request(sql_question: str,
                             params: dict = None,
                             fetch_as_dict: bool = False):
    """
    Executes an asynchronous SQL query, optionally with parameters, and fetches results.
    Uses AsyncDBSession for connection management.
    (Provided by user)
    """
    async with AsyncDBSession() as session:
        try:
            # Use text() for raw SQL queries and bindparams for parameters
            if params:
                result = await session.execute(text(sql_question), params)
            else:
                result = await session.execute(text(sql_question))

            # Check if the result object returns rows before attempting to fetch
            if result.returns_rows:
                if fetch_as_dict:
                    # Fetch all rows as SQLAlchemy Row objects, then convert to dicts
                    rows = result.fetchall()
                    return [row._mapping for row in rows]
                else:
                    # Return list of tuples by default
                    return result.fetchall()
            else:
                # For statements that don't return rows (like DDL, UPDATE, INSERT without RETURNING),
                # we don't call fetchall(). We might want to commit here if this function
                # is intended to commit DDL, but typically DDL is auto-committed or
                # handled by the encompassing transaction.
                # For DROP TABLE, no return value is expected.
                return None # Or a success indicator if needed

        except Exception as e:
            await session.rollback() # Ensure rollback on error
            print(f"Error in open_async_request: {e}")
            raise # Re-raise to propagate the error

In [5]:
await open_async_request("select count(*) from pc_stats")

[(0,)]

In [15]:
async def delete_stats_tables() -> None:
    """
    Deletes the 'gpu_stats' and 'pc_stats' tables from the database.
    This operation is irreversible.
    """

    await open_async_request("DROP TABLE gpu_stats CASCADE;")
    await open_async_request("DROP TABLE pc_stats CASCADE;")
    
    print("Deletion process complete.")

In [20]:
async def delete_all_tables():
    """
    Deletes specified Leela-related tables asynchronously.
    Note: DDL operations like DROP TABLE automatically commit.
    """
    async with AsyncDBSession() as session:
        for table_name_to_delete in ['gpu_stats','pc_stats']:
            print(f"Deleting table: {table_name_to_delete}...")
            try:
                # Use text() for DDL commands
                await session.execute(text(f"DROP TABLE IF EXISTS \"{table_name_to_delete}\" CASCADE;"))
                # No explicit commit needed for DDL in async context as it's auto-committed by the DB
                print(f"Successfully deleted table: {table_name_to_delete}")
            except Exception as e:
                # Rollback is not strictly necessary for DDL that fails, but good general practice
                await session.rollback()
                print(f"An unexpected error occurred during deletion of {table_name_to_delete}: {e}")
                # Don't re-raise immediately if you want to try deleting other tables
                # But for a script, re-raising might be desired for immediate feedback
        # A single commit for the session at the end, though individual DDLs are often auto-committed.
        await session.commit()
    print(f"Tables 'gpu_stats','pc_stats' deleted.")

In [6]:
get_pc_stats()

{'cpu_percent': 0.3,
 'cpu_frequency': {'current': 3696.0, 'min': 0.0, 'max': 0.0},
 'memory': {'total_gb': 27.37,
  'used_gb': 1.49,
  'free_gb': 25.03,
  'percent_used': 7.2},
 'disk_usage': {'total_gb': 1006.85,
  'used_gb': 79.68,
  'free_gb': 875.96,
  'percent_used': 8.3},
 'network_io': {'bytes_sent_mb': 32.35, 'bytes_received_mb': 67.08},
 'gpu_stats': [{'id': 0,
   'name': 'NVIDIA GeForce RTX 4060',
   'load_percent': 26.0,
   'memory_total_mb': 8188.0,
   'memory_used_mb': 2907.0,
   'memory_free_mb': 5051.0,
   'temperature_celsius': 54.0},
  {'id': 1,
   'name': 'NVIDIA GeForce GTX 1070',
   'load_percent': 94.0,
   'memory_total_mb': 8192.0,
   'memory_used_mb': 6253.0,
   'memory_free_mb': 1815.0,
   'temperature_celsius': 80.0}]}